In [1]:
import torch
import numpy as np
import torchvision
from torch import nn
from skimage.transform import resize
from skimage.color import rgb2gray
import pandas as pd 
from torch.nn import Linear, ReLU, BCELoss, Conv2d, Module, Sigmoid
from torch.optim import Adam
from sklearn.metrics import f1_score, precision_score, recall_score
from tqdm import tqdm
import time

In [2]:
torch.backends.cudnn.enabled

True

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.get_device_name(0)

'GeForce 940MX'

In [5]:
def process(df):
    
    #get rid of black images
    df.drop(df[df['Image'].map(np.max) == 0].index,inplace=True)
    
    #get rid of rgb images
    def func(img):
        if len(img.shape)>2:
            return True
        return False
    
    df.drop(df[df['Image'].map(func)].index,inplace=True)
    
    
    #process all the available images; to grayscale,60*60,normalize,proper dimension for pytorch 
    def processImage(px_data):
        
        px_data_scaled = px_data / px_data.max()
        px_data_scaled = resize(px_data, (60, 60), anti_aliasing=True)
        px_data_scaled = px_data_scaled[None,:,:]
        return px_data_scaled    
    
    
    df.loc[:,'Image']=df.apply(lambda x: processImage(x['Image']), axis=1)
    
    #make the labels into bool Alive is True
    df.loc[:,'label'] = df['label'].map(lambda x: x == 'Alive')
    
    return df
    
    

In [6]:
class Dataset(torch.utils.data.Dataset):
    """Pet images dataset."""
    
    @staticmethod
    def from_dataframe(df):
        """
        args:
            petimages : PetImages -- The PetImages object containing the images.
        kwargs:
            size : int -- the size of the canonicalized images.
        """
        data = torch.as_tensor(np.array(df['Image'].tolist(),dtype=np.float32))
        labels = np.array(df['label'],dtype=np.int8)[:,None]

        
        return Dataset(data, labels)

    
    def __init__(self, data, labels):
        # Don't change the constructor
        self.data = data
        self.labels = labels

    def __len__(self):
        """ Return the number of images in the dataset.
        """
        return self.data.shape[0]


    def __getitem__(self, idx):
        """ Return the element corresponding to idx.
        
        args:
            idx : int -- the index of the sample to return
            
        returns: Dict -- A dictionary with two elements; "label" and "image". "label" has the associated label
            and "image" is a (size, size, 3)
        """
        # Convert it to a regular python int.
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        return {'label':self.labels[idx],'image':self.data[idx]}
    
def Dataset_load(df):
    return Dataset.from_dataframe(df)

In [7]:
def split(dataset):
    """ Split pet into train and test sets.
    
    args:
        pet : PetDataset -- the PetDataset instance to split.

    kwargs:
        train_count: The number of elements in the training set. The remainder should be in the test set.
    
    return: List[Dataset] -- the list of [train, test] datasets.
    """
    total = len(dataset)
    test = int(np.ceil(0.2*total))
    train = total-test

    
    return torch.utils.data.random_split(dataset,[train,test])

In [18]:
class Model(nn.Module):
    def __init__(self,num_layers,kernelsize):
        '''num_layers: tuple of size 2, one for each convolutional layer, kernel_size: int'''
        super().__init__()
        self.conv1 = nn.Conv2d(1, num_layers[0], kernelsize)
        self.relu1 = nn.ReLU()
        self.avg1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(num_layers[0], num_layers[1], kernelsize)
        self.tanh2 = nn.Tanh()
        self.avg2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.outconv1 = (60-kernelsize) + 1
        self.outavg1 = (self.outconv1 - 2)/2 + 1
        self.outconv2 = self.outavg1 -kernelsize + 1
        self.outavg2 = (self.outconv2 - 2)/2 + 1
        self.lincount = int(num_layers[1]*self.outavg2**2)
        self.fc1   = nn.Linear(self.lincount, 1)
        self.sigm1 = nn.Sigmoid()
    
    def forward(self, X, debug=False):
        if debug: print(f"Input Shape: {X.shape}")

        X = self.avg1(self.relu1(self.conv1(X)))
        if debug: print(f"Conv1 Shape: {X.shape}")
            
        X = self.avg2(self.tanh2(self.conv2(X)))
        if debug: print(f"Conv1 Shape: {X.shape}")

        X = X.view(X.size(0), -1) # Flatten the shape
        if debug: print(f"Flattened Shape: {X.shape}")

        X = self.sigm1(self.fc1(X))
        if debug: print(f"Output Shape: {X.shape}")

        return X

def count_parameters(model):
    # Count all trainable parameters,
    # from https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/9
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [19]:
mM = Model((30,30),5)
X = torch.zeros(5,1,60,60)
y = mM(X,debug=True)

Input Shape: torch.Size([5, 1, 60, 60])
Conv1 Shape: torch.Size([5, 30, 28, 28])
Conv1 Shape: torch.Size([5, 30, 12, 12])
Flattened Shape: torch.Size([5, 4320])
Output Shape: torch.Size([5, 1])


In [20]:
def training_loop(model, train_dataset, learning_rate = 0.001, epochs=25, batch_size=500):
    """ Train the model on data
    """
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

#     pass # TODO: Set up
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    

    start_time = time.perf_counter()
    for epoch in range(epochs):
        # If you add the training loss to this variable, it will be printed for you
        epoch_loss = 0.0
        
        for data in train_dataloader:
            output = model(data['image'].cuda())
            loss = criterion(output,data['label'].float().cuda())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            epoch_loss+= loss.item()
#         pass # TODO:Process all data for each epoch

        epoch += 1
        if epoch % 50 == 0:
            curr_time = time.perf_counter() - start_time
            print(f'[{curr_time:6.1f}/{curr_time/epoch*epochs:6.1f}] Epoch {epoch: <3d} loss: {epoch_loss / len(train_dataloader)} acc: {test_model(model, train_dataset)}')
    print("Done.")

In [21]:
def test_model(model, test_data, batch_size=500):
    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
    num_correct = 0
    num_alive = 0
    num_alive_correct = 0
    num_dead = 0
    num_dead_correct = 0
    for data in test_dataloader:
        y_pred = model(data["image"].cuda()).round()
        y_actual = data["label"].float().cuda()
        num_correct += (y_pred == y_actual).sum()
        for ya,yp in zip(y_actual,y_pred):
            if ya == 1.0:
                num_alive+=1
                if yp == 1.0:
                    num_alive_correct+=1
            else:
                num_dead+=1
                if yp ==0.0:
                    num_dead_correct+=1
    print(num_dead,num_alive)
    return num_correct.item() / len(test_data),(num_alive_correct/num_alive),(num_dead_correct/num_dead)

In [22]:
def trainModel(filename,num_layers,kernel_size,learning_rate,epoch,batchsize):
    train_df = pd.read_pickle(filename)
    df_train = process(train_df)
    print(len(df_train))
    ds = Dataset_load(df_train)
    train_data, valid_data = tuple(split(ds))

    model = Model(num_layers,kernel_size)
    model.cuda()
    training_loop(model, train_data,learning_rate, epochs = epoch,batch_size = batchsize)

    train_acc,alive_acc,dead_acc = test_model(model, train_data)
    print(f"Train accuracy: {train_acc} Alive accuracy: {alive_acc} Dead accuracy: {dead_acc}")

    test_acc,talive_acc,tdead_acc = test_model(model, valid_data)
    print(f"Test accuracy: {test_acc} Alive accuracy: {talive_acc} Dead accuracy: {tdead_acc}")
    
    return model,[train_acc,alive_acc,dead_acc],[test_acc,talive_acc,tdead_acc]

In [24]:
def hyperparameterOptimization(filename, num_layers,kernelsize,learning_rates, epochs, batch_size = 100):
    max_dead_acc = 0
    best = [(0,0),0,0]
    for num_layer in num_layers:
        for learning_rate in learning_rates:
            for epoch in epochs:
                myModel,train_results,test_results = trainModel("train_csv.pkl",num_layer,kernelsize,learning_rate,epoch,batch_size)
                if test_results[2]>max_dead_acc:
                    max_dead_acc = test_results[2]
                    best = [num_layer,learning_rate,epoch]
    return best


layers = [(12,12),(12,30),(30,30)]
lrs = [0.1,0.01,0.001]
epochs = [100,200]

choices = hyperparameterOptimization("train_csv.pkl", layers,5,lrs, epochs, batch_size = 100)

4345
539 2937
[  96.2/ 192.3] Epoch 50  loss: 4.300051692553929 acc: (0.8449367088607594, 1.0, 0.0)
539 2937
[ 195.0/ 195.0] Epoch 100 loss: 4.300051692553929 acc: (0.8449367088607594, 1.0, 0.0)
Done.
539 2937
Train accuracy: 0.8449367088607594 Alive accuracy: 1.0 Dead accuracy: 0.0
131 738
Test accuracy: 0.8492520138089759 Alive accuracy: 1.0 Dead accuracy: 0.0
4345
538 2938
[  97.8/ 391.2] Epoch 50  loss: 4.2846780776977536 acc: (0.8452243958573072, 1.0, 0.0)
538 2938
[ 197.4/ 394.8] Epoch 100 loss: 4.2846780776977536 acc: (0.8452243958573072, 1.0, 0.0)
538 2938
[ 297.4/ 396.6] Epoch 150 loss: 4.2846780776977536 acc: (0.8452243958573072, 1.0, 0.0)
538 2938
[ 396.3/ 396.3] Epoch 200 loss: 4.2846780776977536 acc: (0.8452243958573072, 1.0, 0.0)
Done.
538 2938
Train accuracy: 0.8452243958573072 Alive accuracy: 1.0 Dead accuracy: 0.0
132 737
Test accuracy: 0.8481012658227848 Alive accuracy: 1.0 Dead accuracy: 0.0
4345
543 2933
[  95.4/ 190.8] Epoch 50  loss: 0.25962415933609007 acc: (0.90

531 2945
[ 553.6/ 738.1] Epoch 150 loss: 4.216950920649937 acc: (0.8472382048331415, 1.0, 0.0)
531 2945
[ 739.0/ 739.0] Epoch 200 loss: 4.216950920649937 acc: (0.8472382048331415, 1.0, 0.0)
Done.
531 2945
Train accuracy: 0.8472382048331415 Alive accuracy: 1.0 Dead accuracy: 0.0
139 730
Test accuracy: 0.8400460299194477 Alive accuracy: 1.0 Dead accuracy: 0.0
4345
531 2945
[ 182.3/ 364.6] Epoch 50  loss: 0.33459288605621884 acc: (0.8757192174913694, 0.9830220713073005, 0.2806026365348399)
531 2945
[ 367.0/ 367.0] Epoch 100 loss: 0.28483943726335254 acc: (0.8955696202531646, 0.9748726655348048, 0.455743879472693)
Done.
531 2945
Train accuracy: 0.8955696202531646 Alive accuracy: 0.9748726655348048 Dead accuracy: 0.455743879472693
139 730
Test accuracy: 0.8814729574223246 Alive accuracy: 0.9671232876712329 Dead accuracy: 0.4316546762589928
4345
538 2938
[ 182.2/ 728.9] Epoch 50  loss: 0.32762346054826463 acc: (0.879746835443038, 0.9833219877467665, 0.3141263940520446)
538 2938
[ 365.7/ 731.

In [25]:
choices

[(12, 12), 0.01, 200]

In [26]:
myModel,tracc,teacc = trainModel("train_csv.pkl",choices[0],5,choices[1],choices[2],100)

8625
1162 5738
[ 186.5/ 746.1] Epoch 50  loss: 0.45215947256572003 acc: (0.8334782608695652, 1.0, 0.011187607573149742)
1162 5738
[ 377.3/ 754.5] Epoch 100 loss: 0.39689004205275275 acc: (0.8508695652173913, 0.9956430812129662, 0.1359724612736661)
1162 5738
[ 567.2/ 756.3] Epoch 150 loss: 0.34303287189939746 acc: (0.8697101449275362, 0.9827466016033461, 0.31153184165232356)
1162 5738
[ 757.9/ 757.9] Epoch 200 loss: 0.3101755320162013 acc: (0.8828985507246376, 0.9872777971418613, 0.3674698795180723)
Done.
1162 5738
Train accuracy: 0.8828985507246376 Alive accuracy: 0.9872777971418613 Dead accuracy: 0.3674698795180723
324 1401
Test accuracy: 0.8220289855072463 Alive accuracy: 0.9493219129193433 Dead accuracy: 0.2716049382716049


In [27]:
test_df = pd.read_pickle('test_data.pkl')
df_test = process(test_df)
print(len(df_test))
ds = Dataset_load(df_test)
test_model(myModel,ds)

2153
394 1759


(0.8392940083604273, 0.9658897100625355, 0.27411167512690354)